In [4]:
import PIL
import imageio
import torchvision
import torch
import cv2
from torchvision.transforms import transforms
# from detector.detect2 import Detector
# from detector.productdetector import draw_rectange, put_text
from train.dataset import ProductsDataset
import pickle
import pandas as pd
from pathlib import Path

In [5]:
csv_folder = "csv/"
df = pd.read_csv(csv_folder + "fruits_vegetables_filtered.csv")
products_types_dict = dict(zip(df["Материал"], df["Вид"]))
df = pd.read_csv(csv_folder + "main_products.csv")
products_types_dict_main = dict(zip(df["Материал"], df["Classifier"]))
products_types_dict.update(products_types_dict_main)
model = "efficientnet_b3"
num_classes = 211  # TODO read from config
model = torchvision.models.__dict__[model](num_classes=num_classes)
gpu_flag = 0
device = torch.device("cuda" if gpu_flag else "cpu")
model.load_state_dict(
    torch.load("train/checkpoint.pth", map_location=device)["model"]
)
model.eval()
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

trans = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=mean, std=std),
    ]
)
with open("dataset/classes.pkl", "rb") as f:
    class_to_idx = pickle.load(f)
idx_to_cls = {idx: cls for cls, idx in class_to_idx.items()}

In [ ]:
root = Path("dataset/val")
count = 0
total = 0
with torch.inference_mode():
    for d1 in root.iterdir():
        for d2 in d1.iterdir():
            for d3 in d2.iterdir():
                for plu_dir in d3.iterdir():
                    for path_obj in plu_dir.iterdir():
                        crop = PIL.Image.open(path_obj)
                        res = model.forward(torch.unsqueeze(trans(crop), 0))[0].argmax()
                        # print(idx_to_cls[int(res)], products_types_dict[int(plu_dir.name)])
                        plu = int(plu_dir.name)
                        cls = "Other" if plu not in products_types_dict else products_types_dict[plu]
                        print(cls, idx_to_cls[int(res)])
                        # count += int(res) == class_to_idx[cls]
                        # total += 1
print(count/total)